In [2]:
#imports 
import torch as th
import syft as sy

In [3]:
#create a hook 
hook = sy.TorchHook(th)

In [4]:
#making the data sets 
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [5]:
#import for the model 
from torch import nn, optim

In [7]:
#makig the toy model 
model = nn.Linear(2,1)

In [8]:
#crearting a optimizer 
opt = optim.SGD(params=model.parameters(),lr = 0.1)

In [24]:
def train():
    for i in range(20):


        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)** 2).sum()
        loss.backward()
        opt.step()
        print(loss.data)

train()

tensor(0.0023)
tensor(0.0017)
tensor(0.0012)
tensor(0.0009)
tensor(0.0007)
tensor(0.0005)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(9.0949e-05)
tensor(6.8965e-05)
tensor(5.2363e-05)
tensor(3.9802e-05)
tensor(3.0283e-05)
tensor(2.3059e-05)
tensor(1.7570e-05)
tensor(1.3395e-05)
tensor(1.0217e-05)


In [25]:
#make the virtual worker in the cloud
#create a worker 
bob = sy.VirtualWorker(hook, id ="bob")
alice = sy.VirtualWorker(hook,id = "alice")

In [32]:
#sending the data to the worker bob
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [33]:
#sending the data to the Alice
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [34]:
#making the data tuples 
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [36]:
#make the model
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [37]:
#viusuaizing the dataset 
_data, _target = datasets[0]

In [38]:
_data

(Wrapper)>[PointerTensor | me:44487894627 -> bob:29157150187]

In [49]:
def train(iterations = 20):
    #make the model
    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(iterations):
        for _data, _target in datasets:

            #sending the model to the virtual worker 
            model = model.send(_data.location)

            # do the normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred- _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back 
            model = model.get()


            print(loss.get())

In [50]:
train()

tensor(0.1067, requires_grad=True)
tensor(0.0417, requires_grad=True)
tensor(0.0343, requires_grad=True)
tensor(0.0286, requires_grad=True)
tensor(0.0238, requires_grad=True)
tensor(0.0198, requires_grad=True)
tensor(0.0165, requires_grad=True)
tensor(0.0137, requires_grad=True)
tensor(0.0114, requires_grad=True)
tensor(0.0095, requires_grad=True)
tensor(0.0079, requires_grad=True)
tensor(0.0066, requires_grad=True)
tensor(0.0055, requires_grad=True)
tensor(0.0046, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0032, requires_grad=True)
tensor(0.0026, requires_grad=True)
tensor(0.0022, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0013, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0003, requi